In [141]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import datpiffUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from ioUtils import getFile, saveFile
from fileUtils import getFileBasics, getBasename
from webUtils import getWebData, getHTML, getURL
from artistDP import artistDP
from artistsDP import artistsDP
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-03-29 20:31:18.501361


# Initialize

In [142]:
%load_ext autoreload
%autoreload
discDP = discogs('datpiff')
dutils = datpiffUtils()
artDP  = artistDP()
artsDP = artistsDP(discDP)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Download Known Artists

In [ ]:
def getMyMusicDBMap():
    myMusicMap = getFile(ifile="/Users/tgadfort/Documents/code/music/myMusicMap.p")
    print("Found {0} entries".format(len(myMusicMap)))
    return myMusicMap

myMusicMap = getMyMusicDBMap()

In [136]:
%load_ext autoreload
%autoreload
discDP = discogs('datpiff')
dutils = datpiffUtils()
artDP  = artistDP()
artsDP = artistsDP(discDP)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
artsDP.searchRateYouDPusicForArtist("Yo Gotti")

In [43]:
from multiArtist import multiartist
mulArts  = multiartist()

In [64]:
known = ["Yo Gotti", "Gucci Mane"]
known = getFile(ifile="datPiffKnown.p")
#saveFile(idata=known, ifile="/Users/tgadfort/Music/Discog/artists-datpiff/knownSearches.p")
known = getFile(ifile="/Users/tgadfort/Music/Discog/artists-datpiff/knownSearches.p")
print("There are {0} searched terms".format(len(known)))

There are 402 searched terms


In [139]:
from collections import Counter
from time import sleep
from glob import glob

artistsCntr = Counter()
for ifile in glob("/Users/tgadfort/Music/Discog/artists-datpiff/*/*.p"):
    tmp     = getFile(ifile)
    results = [x["ArtistName"] for x in tmp]
    for artist in results:
        artists = mulArts.getArtistNames(artist)
        for artist in artists.keys():
            key = artist.title()
            if len(key) > 1 and key not in known:
                artistsCntr[key] += 1
searchTerms = [item[0] for item in artistsCntr.most_common() if item[1] >= 40]
print("There are {0} new searches".format(len(searchTerms)))

There are 124 new searches


In [140]:
for i,searchTerm in enumerate(searchTerms):
    print("\n=====>",i,"/",len(searchTerms),"==>",searchTerm)
    artsDP.searchRateYouDPusicForArtist(searchTerm)
    known.append(searchTerm)
    sleep(3)


=====> 0 / 124 ==> C-Bo


===================== Searching For C-Bo =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=C-Bo&sort=relevance
Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/5/3241105.p
  --> This file is 5.9kB.

=====> 1 / 124 ==> L.I.


===================== Searching For L.I. =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=L.I.&sort=relevance
Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/84/3885184.p
  --> This file is 3.0kB.

=====> 2 / 124 ==> Mac Dre


===================== Searching For Mac Dre =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Mac%20Dre&sort=relevance
Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/79/1795679.p
  --> This file is 5.9kB.

=====> 3 / 124 ==> Lee Majors


===================== Searching For Lee Majors =====================
	Search URL: https://www.datpiff.com/mixtapes-searc


=====> 28 / 124 ==> Chippass


===================== Searching For Chippass =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Chippass&sort=relevance
Saving 6 new artist media to /Volumes/Biggy/Discog/artists-datpiff/25/74725.p
  --> This file is 1.1kB.

=====> 29 / 124 ==> Laroo


===================== Searching For Laroo =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Laroo&sort=relevance
Saving 11 new artist media to /Volumes/Biggy/Discog/artists-datpiff/48/6028648.p
  --> This file is 1.7kB.

=====> 30 / 124 ==> Onyx


===================== Searching For Onyx =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Onyx&sort=relevance
Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/76/2371976.p
  --> This file is 5.6kB.

=====> 31 / 124 ==> Alison Hinds


===================== Searching For Alison Hinds =====================
	Search URL: https://www.datpiff.com/mixt

Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/56/1062356.p
  --> This file is 4.2kB.

=====> 57 / 124 ==> Rapsody


===================== Searching For Rapsody =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Rapsody&sort=relevance
Saving 17 new artist media to /Volumes/Biggy/Discog/artists-datpiff/78/9995278.p
  --> This file is 2.2kB.

=====> 58 / 124 ==> Roc Marciano


===================== Searching For Roc Marciano =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Roc%20Marciano&sort=relevance
Saving 31 new artist media to /Volumes/Biggy/Discog/artists-datpiff/45/1398745.p
  --> This file is 4.5kB.

=====> 59 / 124 ==> Slum Village


===================== Searching For Slum Village =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Slum%20Village&sort=relevance
Saving 30 new artist media to /Volumes/Biggy/Discog/artists-datpiff/16/199616.p
  --> This file is 


=====> 85 / 124 ==> Vince Staples


===================== Searching For Vince Staples =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Vince%20Staples&sort=relevance
Saving 34 new artist media to /Volumes/Biggy/Discog/artists-datpiff/83/3239683.p
  --> This file is 4.1kB.

=====> 86 / 124 ==> Ice Berg


===================== Searching For Ice Berg =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Ice%20Berg&sort=relevance
Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/39/2625939.p
  --> This file is 4.4kB.

=====> 87 / 124 ==> Marcia Griffiths


===================== Searching For Marcia Griffiths =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Marcia%20Griffiths&sort=relevance
Saving 7 new artist media to /Volumes/Biggy/Discog/artists-datpiff/66/6138366.p
  --> This file is 1.3kB.

=====> 88 / 124 ==> Ester Dean


===================== Searching For Ester Dean

Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/81/3018381.p
  --> This file is 5.6kB.

=====> 113 / 124 ==> The Outlawz


===================== Searching For The Outlawz =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=The%20Outlawz&sort=relevance
Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/64/6494364.p
  --> This file is 5.9kB.

=====> 114 / 124 ==> Cali Swag District


===================== Searching For Cali Swag District =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Cali%20Swag%20District&sort=relevance
Saving 27 new artist media to /Volumes/Biggy/Discog/artists-datpiff/35/2020335.p
  --> This file is 4.0kB.

=====> 115 / 124 ==> Vegas


===================== Searching For Vegas =====================
	Search URL: https://www.datpiff.com/mixtapes-search?criteria=Vegas&sort=relevance
Saving 52 new artist media to /Volumes/Biggy/Discog/artists-datpiff/87/5719987.p
  --

In [143]:
print("Saving {0} known search terms".format(len(known)))
saveFile(idata=known, ifile="datPiffKnown.p")
saveFile(idata=known, ifile="/Users/tgadfort/Music/Discog/artists-datpiff/knownSearches.p")

Saving 1120 known search terms
  --> This file is 10.7kB.
  --> This file is 10.7kB.


In [137]:
artsDP.parseArtistFiles()

Found 995 downloaded search terms
Saving 406 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/19-DB.p
  --> This file is 520.1kB.
Saving 406 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/19-Metadata.p
  --> This file is 11.4kB.
Saving 406 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/19-MediaMetadata.p
  --> This file is 30.6kB.
Saving 420 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/82-DB.p
  --> This file is 361.8kB.
Saving 420 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/82-Metadata.p
  --> This file is 12.0kB.
Saving 420 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/82-MediaMetadata.p
  --> This file is 31.8kB.
Saving 380 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/0-DB.p
  --> This file is 1.0MB.
Saving 380 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/0-Metadata.p
  --> This file is 10.5kB.
Sa

  --> This file is 438.0kB.
Saving 422 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/64-Metadata.p
  --> This file is 12.0kB.
Saving 422 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/64-MediaMetadata.p
  --> This file is 31.3kB.
Saving 400 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/2-DB.p
  --> This file is 349.2kB.
Saving 400 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/2-Metadata.p
  --> This file is 11.2kB.
Saving 400 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/2-MediaMetadata.p
  --> This file is 29.9kB.
Saving 440 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/54-DB.p
  --> This file is 449.0kB.
Saving 440 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/54-Metadata.p
  --> This file is 12.0kB.
Saving 440 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/54-MediaMetadata.p


  --> This file is 541.8kB.
Saving 424 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/67-Metadata.p
  --> This file is 11.7kB.
Saving 424 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/67-MediaMetadata.p
  --> This file is 31.1kB.
Saving 401 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/83-DB.p
  --> This file is 430.1kB.
Saving 401 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/83-Metadata.p
  --> This file is 11.0kB.
Saving 401 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/83-MediaMetadata.p
  --> This file is 29.6kB.
Saving 422 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/92-DB.p
  --> This file is 348.9kB.
Saving 422 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/92-Metadata.p
  --> This file is 11.7kB.
Saving 422 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/92-MediaMetadata

  --> This file is 326.4kB.
Saving 409 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/45-Metadata.p
  --> This file is 11.3kB.
Saving 409 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/45-MediaMetadata.p
  --> This file is 30.6kB.
Saving 406 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/32-DB.p
  --> This file is 289.7kB.
Saving 406 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/32-Metadata.p
  --> This file is 11.3kB.
Saving 406 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/32-MediaMetadata.p
  --> This file is 30.3kB.
Saving 399 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/91-DB.p
  --> This file is 746.2kB.
Saving 399 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/91-Metadata.p
  --> This file is 11.1kB.
Saving 399 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/91-MediaMetadata

  --> This file is 361.3kB.
Saving 375 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/18-Metadata.p
  --> This file is 10.5kB.
Saving 375 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/18-MediaMetadata.p
  --> This file is 28.3kB.
Saving 424 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/28-DB.p
  --> This file is 394.0kB.
Saving 424 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/28-Metadata.p
  --> This file is 11.8kB.
Saving 424 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/28-MediaMetadata.p
  --> This file is 32.1kB.
Saving 447 artist IDs to /Volumes/Biggy/Discog/artists-datpiff-db/65-DB.p
  --> This file is 590.4kB.
Saving 447 new artist IDs name data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/65-Metadata.p
  --> This file is 12.2kB.
Saving 447 new artist IDs media data to /Volumes/Biggy/Discog/artists-datpiff-db/metadata/65-MediaMetadata

In [ ]:
def downloadSearchResults(maxI=-1):
    for i,artistName in enumerate(myMusicMap.keys()):
        if i <= maxI:
            continue
        print("====>",i,"/",len(myMusicMap),"  : ",artistName)
#        try:
        artsLM.searchLastFMForArtist(artistName, sleeptime=3)
#        except:
#            sleep(3)
        sleep(3)

In [ ]:
downloadSearchResults(3544)

# Download Extra Artist Files

In [ ]:
artsLM.assertDBModValExtraData(61)

In [ ]:
retval = artLM.getData("/Volumes/Biggy/Discog/artists-lastfm/51/34806831451.p")

In [ ]:
artsLM.parseArtistFiles()

In [ ]:
dutils.getArtistID("Coldplay vs. The Killers vs. Kelly Clarkson vs. Bon Jovi vs. Empire Of The Sun vs. Muse")  # 99596595167
dutils.getArtistID("Coldplay vs. The Killers vs. Kelly Clarkson vs. Bon…")   # 93496498096

In [ ]:
url="https://www.last.fm/search/artists?q=Coldplay"
savename="lm_search.p"
artsLM.downloadArtistURL(url=url, savename=savename)

In [ ]:
bsdata = getHTML("lm_search.p")

In [ ]:
uls = bsdata.findAll("ul", {"class": "artist-results"})
for ul in uls:
    lis = ul.findAll("li", {"class": "artist-result"})
    for li in lis:
        h4 = li.find("h4")
        if h4 is None:
            raise ValueError("No h4 in list")
        result   = artLM.getNamesAndURLs(h4)
        name     = result[0].name
        url      = result[0].url
        artistID = dutils.getArtistID(name)
        print(name,'\t',url,'\t',artistID)

In [ ]:
#bsdata

In [ ]:
retval = artLM.getData("lastfm.p")

In [ ]:
retval.artist.get()

In [ ]:
retval.pages.get()

In [ ]:
retval.url.get()

In [ ]:
retval.ID.get()

In [ ]:
retval.profile.get()

In [ ]:
for album in retval.media.media["Albums"]:
    print(album.get())

In [ ]:
retval.mediaCounts.get()

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs('lastfm')
arts = artists(disc)
#arts.searchDiscogForArtist("The Whispers")

In [ ]:
from webUtils import getWebData
url="https://www.last.fm/music/Coldplay/+albums"
url="https://www.last.fm/music/Coldplay/+albums?order=release_date"
url="https://www.last.fm/music/Coldplay/+albums?order=release_date"
savename="lastfm.p"
getWebData(base=url, savename=savename, useSafari=False)

In [ ]:
url="https://www.datpiff.com/mixtapes-search?criteria=yo%20gotti&sort=relevance"
savename = "dp_search.p"
getWebData(base=url, savename=savename, useSafari=False)

In [ ]:
getWebData

In [ ]:
bsdata = getHTML("dp_search.p")

In [ ]:
bsdata

In [ ]:
contentdivs = bsdata.findAll("div", {"class": "contentItem"})
for i,contentdiv in enumerate(contentdivs):
    artistDiv = contentdiv.find("div", {"class": "artist"})
    if artistDiv is None:
        raise ValueError("There is no artist div")
    artistName = artistDiv.text
    
    albumDiv = contentdiv.find("div", {"class": "title"})
    if albumDiv is None:
        raise ValueError("There is no title div")
    albumName = albumDiv.text
    try:
        albumURL  = albumDiv.find("a").attrs['href']
    except:
        albumURL  = None
        
    print(artistName,'\t',albumName)

In [ ]:
divs[1]